In [1]:
import random
from tqdm import tqdm
import pandas as pd
from prompt_design import design0
import pandas as pd
import re
data_dir = 'generation_data2/'


Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-char were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
use Dict normalizer


In [2]:
df_org = pd.read_csv('../data/tweet_hyakuyaku_processed.csv')

In [3]:
df_org['query'].unique()

array(['ステロイド', 'インスリン', '抗生剤', 'プレドニゾロン', '抗菌薬', 'ヘパリン', '利尿剤', 'プレドニン',
       '補液', 'ステロイドパルス', 'ワーファリン', 'アルドステロン', 'コルチゾール', 'シクロスポリン',
       '免疫抑制剤', 'アルブミン', 'アミオダロン', 'ＳＴ合剤', '利尿薬', 'カテコラミン', '抗結核薬',
       '抗生物質', 'メチルプレドニゾロン', 'アスピリン', '造影剤', 'フロセミド', 'デキサメサゾン', 'ワルファリン',
       'アシクロビル', 'タクロリムス', 'リツキシマブ', '降圧薬', 'ヒドロコルチゾン', 'メサラジン',
       '免疫グロブリン', 'チアマゾール', '抗血小板薬', 'イマチニブ', 'グルカゴン', 'γグロブリン', 'トルバプタン',
       '強化インスリン', '降圧剤', 'インターフェロン', 'インフリキシマブ', 'シクロホスファミド', 'メトトレキサート',
       'メロペネム', 'メトホルミン', 'バンコマイシン', 'ミノサイクリン', 'アザチオプリン', 'シスプラチン',
       'ウロキナーゼ', 'グリメピリド', 'デキサメタゾン', 'セフトリアキソン', 'メトロニダゾール', '抗菌剤', '輸液',
       'リバビリン', 'アルガトロバン', 'シクロフォスファミド', 'ガンシクロビル', 'ステロイド剤', 'ノルアドレナリン',
       '抗凝固薬', 'コルヒチン'], dtype=object)

In [7]:
df_org.head()

,Unnamed: 0,full_text,query
0,1,'@cyocyo_cyocyo 今回のラグオリパスでちょっとやる気が出たからぼちぼちやってい...,ステロイド
1,7,'SAIDs 出来【デキサメサゾン】杉くんの、ゆめ散る【メチルプレドニゾロン】プレ【プレドニ...,ステロイド
2,10,'それに、ステロイドで酒さになってるのになんでステロイドだすの、お医者さん、、。',ステロイド
3,14,'年明け早々、グリーンピールするか皮膚科でも1回みてもらうか迷う| ε:)_\n去年はグリー...,ステロイド
4,15,'いわゆるアメコミのヒーローらは全てが『筋骨隆々』であり、星条旗の前で逞しい胸を張るスーパー...,ステロイド


In [8]:
prompt_list, target_list = design0(df_org)
df4data = pd.DataFrame({'Target':target_list,'Prompt':prompt_list})
df4data.to_csv(data_dir+"data.csv",index = None, header=True)

In [ ]:
prompt_list[0]

In [29]:
df4data['Target'] = df4data['Target'].map(lambda x:re.sub(r'@[0-9a-zA-Z_:]*','<user_name>',x))

In [32]:
df4data.to_csv(data_dir+'data.csv')

In [33]:
all_data = []

In [34]:
t_s = zip(list(df4data['Target']),list(df4data['Prompt']))
for t,s in t_s:
    data = {}
    if isinstance(t,str) and isinstance(s,str):
        data['Target'] = t.replace('\n',' ')
        data['Prompt'] = s.replace('\n',' ')
        all_data.append(data)    
    else:
        pass

In [35]:
import random
random.seed(1234)
random.shuffle(all_data)

def to_line(data):
    Target = data["Target"]
    prompt = data["Prompt"]
    assert len(Target) > 0 and len(prompt)
    return f"{Target}\t{prompt}\n"

In [36]:
data_size = len(all_data)
train_ratio, dev_ratio, test_ratio = 0.9, 0.05, 0.05
with open(data_dir+"train.tsv", "w", encoding="utf-8") as f_train, \
open(data_dir+"dev.tsv", "w", encoding="utf-8") as f_dev, \
    open(data_dir+"test.tsv", "w", encoding="utf-8") as f_test:
    for i, data in tqdm(enumerate(all_data)):
        line = to_line(data)
        if i < train_ratio * data_size:
            f_train.write(line)
        elif i < (train_ratio + dev_ratio) * data_size:
            f_dev.write(line)
        else:
            f_test.write(line)

441152it [00:01, 371386.51it/s]


In [ ]:
from medner_j import Ner

sents = [
  "それぞれの関節に関節液貯留は見られなかった",
  "その後、左半身麻痺、ＣＴにて右前側頭葉の出血を認める。"
  ]

model = Ner.from_pretrained(model_name="BERT", normalizer="dict")
results = model.predict(sents)
print(results)

In [ ]:
results2 = model.predict(sents)
print(results2)

In [ ]:
'</C>' in results2[1] or '</CN>' in results2[1]

In [19]:
df4data.head()

,Target,Prompt
0,<extra_id_0>'No.1-1\n・16歳未満\n・原因不明の慢性関節炎\n・弛緩熱...,ステロイド使用のTweetは？<extra_id_0>
1,<extra_id_0>'ステロイド服用してるけど食欲が本当に増える。浮腫みで太るって人いる...,ステロイド使用のTweetは？<extra_id_0>
2,<extra_id_0>'9時から参拝、10時からバイトだけど朝までアレルギーで鼻ずるずる、...,ステロイド使用のTweetは？<extra_id_0>
3,<extra_id_0>'ステロイドを塗ると皮膚が黒くなるの？\n皮膚が色素沈着を起こすのは...,ステロイド使用のTweetは？<extra_id_0>
4,<extra_id_0>'唇の端が切れたりする症状である口角炎は、ビタミン不足や細菌やカビの...,ステロイド使用のTweetは？<extra_id_0>
